# Fluke tseting flow

In [29]:
from  scapy.all import *
import netifaces
from subprocess import run

print ("Testing...") 

'''LINK STATUS'''
carrier=open('/sys/class/net/eth0/carrier','r') 
connect=carrier.readline()
if (connect=='0\n'):
    print ("operstate: down")
else:
    print ("operstate: up")
        
    '''IP state'''
    interface=open('/etc/network/interfaces','r')
    state=interface.readlines()
    static=False
    for i in state:
        if (0<i.find("static")):
            static=True
    if (static==True):        
        print("IP state: Static")
    else:
        print("IP state: DHCP")
            
    '''print network_inf'''
    try:
        addr=netifaces.ifaddresses('eth0')[2][0]['addr']
        netmask=netifaces.ifaddresses('eth0')[2][0]['netmask']
        gateway=netifaces.gateways()[2][0][0]
        network_inf = "Address: %s\nNetmask: %s\nGateway: %s"%(addr,netmask,gateway)
        print (network_inf)
    except:
        print ('no information about network')
        
    print ("Ping Gateway...")
    try:
        ping_gw='ping '+gateway+" -c 10"
        gw_test=subprocess.getoutput(ping_gw)
        ping_gw_result=(gw_test.split(','))[1]
        ping_gw_result=int(ping_gw_result.split(' ')[1])
    except:
        pass
    
    if (ping_gw_result>=9):
        print ("Sucess")
    else:
        print ("Failed")
print("Finished Test.")

Testing...
operstate: up
IP state: DHCP
no information about network
Ping Gateway...
Sucess
Finished Test.


# Catch DATA

In [ ]:
from  scapy.all import *
catch =sniff (iface='eth0',filter ="ip and udp port 1900 or arp" ,count=2)
print (catch)
catch.show()

In [7]:
lenth=len(catch)
for i in range(lenth):
        if (int(catch[i][0].type)==2048):
            print (catch[i][IP].src)
        else:
            print ("ARP")
            print (catch[i][ARP].psrc)

ARP
120.101.8.229
ARP
120.101.8.86
120.101.8.116
120.101.8.149
120.101.8.229
120.101.8.229
ARP
120.101.8.86
120.101.8.229
120.101.8.149
120.101.8.199
ARP
120.101.8.52
120.101.8.199
ARP
120.101.8.74
120.101.8.229
120.101.8.229
ARP
120.101.8.254
120.101.8.199
120.101.8.229
ARP
120.101.8.38
120.101.8.189
120.101.8.199
120.101.8.189
120.101.8.189
120.101.8.189
ARP
120.101.8.122
ARP
120.101.8.254
ARP
120.101.8.254
ARP
120.101.8.107
ARP
120.101.8.254
ARP
120.101.8.88
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
192.168.0.254
ARP
120.101.8.254
ARP
120.101.8.95
ARP
120.101.8.254
120.101.8.150
ARP
120.101.8.107
120.101.8.150
120.101.8.150
ARP
120.101.8.254
120.101.8.150
120.101.8.106
120.101.8.123
120.101.8.123
120.101.8.123
120.101.8.123
120.101.8.123
120.101.8.123
ARP
120.101.8.254
120.101.8.106
120.101.8.231
120.101.8.106
120.101.8.231
120.101.8.106
120.101.8.231
120.101.8.96
120.101.8.231
120.101.8.96
ARP
120.1

In [9]:
import ipaddress
lenth=len(catch)
data=[]
num=[]
for i in range(lenth):
    if (int(catch[i][0].type)==2048):
        if (catch[i][IP].src!="0.0.0.0"):
            tmp=catch[i][IP].src
            data.append(tmp)
            num.append(int(ipaddress.ip_address(tmp)))
    elif (int(catch[i][0].type)==2054):
        if (catch[i][ARP].psrc!="0.0.0.0"):
            tmp=catch[i][ARP].psrc
            data.append(tmp)
            num.append(int(ipaddress.ip_address(tmp)))
print (data)
print (num)

['120.101.8.229', '120.101.8.86', '120.101.8.116', '120.101.8.149', '120.101.8.229', '120.101.8.229', '120.101.8.86', '120.101.8.229', '120.101.8.149', '120.101.8.199', '120.101.8.52', '120.101.8.199', '120.101.8.74', '120.101.8.229', '120.101.8.229', '120.101.8.254', '120.101.8.199', '120.101.8.229', '120.101.8.38', '120.101.8.189', '120.101.8.199', '120.101.8.189', '120.101.8.189', '120.101.8.189', '120.101.8.122', '120.101.8.254', '120.101.8.254', '120.101.8.107', '120.101.8.254', '120.101.8.88', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '192.168.0.254', '120.101.8.254', '120.101.8.95', '120.101.8.254', '120.101.8.150', '120.101.8.107', '120.101.8.150', '120.101.8.150', '120.101.8.254', '120.101.8.150', '120.101.8.106', '120.101.8.123', '120.101.8.123', '120.101.8.123', '120.101.8.123', '120.101.8.123', '120.101.8.123', '120.101.8.254', '120.101.8.106', '120.101.8.231', '12

# List to Dataframe

In [10]:
import pandas as pd
data={'number':num,'ip_address':data}
data=pd.DataFrame(data)

In [11]:
data3=data.drop_duplicates('ip_address',keep='first',inplace=False)
data3=data3.sort_values(by='number')
data3

In [13]:
last=(data3.iloc[-3]).ip_address

In [14]:
first=(data3.iloc[0]).ip_address

In [15]:
lasto=int(ipaddress.IPv4Address(last))
firsto=int(ipaddress.IPv4Address(first))
mask=((lasto&firsto)|(~(lasto|firsto)))
print (mask)
tmp=int(ipaddress.IPv4Address("255.255.255.255"))+mask
print (tmp)
#mask=str(ipaddress.IPv4Address((lasto^firsto)))
tmp2=str(ipaddress.IPv4Address(tmp))
print (tmp2)

-194
4294967101
255.255.255.61


# Network mask 

In [16]:
#print (ipaddress.IPv4Address(tmp2))
for i in range(2,31):
    temp=i
    temp=str(temp)
    s1="255.255.255.0/" + temp
    z=ipaddress.ip_network(s1,strict=False)
    #print (z)
    mask=0
    if(ipaddress.IPv4Address(tmp2)<z.netmask):
        print (i-1)
        mask=i-1
        break

24


# Gateway

In [17]:
arp=[]
for i in range(lenth):
    if (int(catch[i][0].type)==2054):
        if (catch[i][ARP].psrc!="0.0.0.0"):
            src=catch[i][ARP].psrc
            dst=catch[i][ARP].pdst
            arp.append(src)
            arp.append(dst)
print (arp)

['120.101.8.229', '120.101.8.92', '120.101.8.86', '120.101.8.215', '120.101.8.86', '120.101.8.215', '120.101.8.52', '120.101.8.254', '120.101.8.74', '120.101.8.1', '120.101.8.254', '120.101.8.153', '120.101.8.38', '120.101.8.254', '120.101.8.122', '120.101.8.96', '120.101.8.254', '120.101.8.184', '120.101.8.254', '120.101.8.15', '120.101.8.107', '120.101.8.86', '120.101.8.254', '120.101.8.15', '120.101.8.88', '120.101.8.1', '120.101.8.254', '120.101.8.12', '120.101.8.95', '120.101.8.254', '120.101.8.254', '120.101.8.95', '120.101.8.107', '120.101.8.141', '120.101.8.254', '120.101.8.165', '120.101.8.254', '120.101.8.248', '120.101.8.254', '120.101.8.15', '120.101.8.254', '120.101.8.15', '120.101.8.254', '120.101.8.246']


In [29]:
dict={}
maxaddr=0
GW=''
for key in arp:
    dict[key]=dict.get(key,0)+1
    if (maxaddr<dict[key]):
        maxaddr=dict[key]
        GW=key
print(dict)

{'120.101.8.229': 1, '120.101.8.92': 1, '120.101.8.86': 3, '120.101.8.215': 2, '120.101.8.52': 1, '120.101.8.254': 14, '120.101.8.74': 1, '120.101.8.1': 2, '120.101.8.153': 1, '120.101.8.38': 1, '120.101.8.122': 1, '120.101.8.96': 1, '120.101.8.184': 1, '120.101.8.15': 4, '120.101.8.107': 2, '120.101.8.88': 1, '120.101.8.12': 1, '120.101.8.95': 2, '120.101.8.141': 1, '120.101.8.165': 1, '120.101.8.248': 1, '120.101.8.246': 1}


In [32]:
dict['120.101.8.254']

14

In [33]:
GW

'120.101.8.254'